## Testing hypothesis 3 in a Big Data context using PySpark
---

**_Hypothesis_**: Reviews with higher book rating have higher helpfulness ratings.

   - **Metric**: Correlation between helpfulness_score book ratings.

**Missing Values**:

  - `review/score`: remove the entire sample
  - `Tot_votes`: remove the entire sample

**Data Transformation**:
  - `review/score`: groupBy book ratings and calculate the average score.
  - `review/helpfulness`: $helpfulness = \frac{x}{y} \sqrt(y)$

---

In [ ]:
# Initialize spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("hypothesis_3").getOrCreate()

In [ ]:
# Import libraries
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt


In [ ]:
# Define the schema
ratings_schema = StructType([
    StructField("Title", StringType(), True),
    StructField("Price", FloatType(), True),
    StructField("User_id", IntegerType(), True),
    StructField("profileName", StringType(), True),
    StructField("review/score", FloatType(), True),
    StructField("review/time", IntegerType(), True),
    StructField("review/summary", StringType(), True),
    StructField("review/text", StringType(), True),
    StructField("N_helpful", IntegerType(), True),
    StructField("Tot_votes", IntegerType(), True)
])

# Load the data
df_ratings = spark.read.csv('hdfs://localhost:9900/user/book_reviews/books_rating_cleaned.csv', header=True, schema=ratings_schema, sep='\t')
df_ratings.show(5)

In [ ]:
# Filter out the data
df_ratings_filtered = df_ratings.filter(df_ratings['review/score'].isNotNull())
df_ratings_filtered = df_ratings_filtered.filter(df_ratings_filtered['Tot_votes'] > 20)

# Add the helpfulness ratio column
df_ratings_filtered = df_ratings_filtered.withColumn('helpfulness_ratio', df_ratings_filtered['N_helpful']/df_ratings_filtered['Tot_votes']*sqrt(df_ratings_filtered['Tot_votes']))

# Retain only the required columns
df_ratings_selected = df_ratings_filtered.select('review/score', 'helpfulness_ratio')

# Compute the spearman correlation (it requires Mlib to work with DataFrame API)
vector_col = VectorAssembler(inputCols=df_ratings_selected.columns, outputCol='features')
dataset = vector_col.transform(df_ratings_selected).select('features')
corr = Correlation.corr(dataset, 'features', method='spearman').collect()[0][0].toArray()[0][1]
print(f'The Spearman correlation coefficient is {round(corr,2)}')

In [ ]:
# Plot the results
df_grouped = df_ratings_selected.groupBy('review/score').agg(avg('helpfulness_ratio'))
df_grouped = df_grouped.sort('review/score')
#df_grouped.plot.bar(x='review/score', y='avg(helpfulness_ratio)', rot=0, title='Average helpfulness ratio per score')

df_pandas = df_grouped.toPandas()
plt.figure(figsize=(10,5))
df_pandas.plot.bar(x='review/score', y='avg(helpfulness_ratio)', rot=0, title='Average helpfulness ratio per score', legend = False)
